In [3]:
import re

from dotenv import load_dotenv
import pandas as pd
import tiktoken
# LangChain imports
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_google_genai import ChatGoogleGenerativeAI


# Analise de Desemepnho de LLM

In [ ]:

import re

def parse_messages_from_text(full_text):
    full_text = full_text.replace('\u200e', '')  # remove caractere invisível comum no WhatsApp

    # Regex robusto para capturar cada mensagem (com quebra de linha dentro do conteúdo)
    pattern = r'''
        ^\[(\d{2}/\d{2}/\d{2}),\s(\d{2}:\d{2}:\d{2})\]    # data e hora no início da linha
        \s(.+?):                                          # autor (termina no primeiro :)
        \s(.*?)                                           # conteúdo da mensagem
        (?=^\[\d{2}/\d{2}/\d{2},\s\d{2}:\d{2}:\d{2}\]|\Z)  # lookahead: próxima msg ou fim do texto
    '''

    matches = re.finditer(pattern, full_text, re.DOTALL | re.MULTILINE | re.VERBOSE)

    structured = []
    spans = []

    for match in matches:
        spans.append(match.span())
        structured.append({
            'date': match.group(1),
            'time': match.group(2),
            'author': match.group(3).strip(),
            'content': match.group(4).strip()
        })

    # Extrair o texto que ficou de fora (não estruturado)
    unstructured = []
    last_end = 0
    for start, end in spans:
        if start > last_end:
            chunk = full_text[last_end:start].strip()
            if chunk:
                unstructured.append(chunk)
        last_end = end

    if last_end < len(full_text):
        chunk = full_text[last_end:].strip()
        if chunk:
            unstructured.append(chunk)

    return structured, unstructured



def contar_tokens(texto, modelo="gpt-4"):
    tokenizer = tiktoken.encoding_for_model(modelo)
    tokens = tokenizer.encode(texto)
    return len(tokens)


In [23]:
final = None
with open('_chat.txt', 'r', encoding='utf-8') as arquivo:
    conteudo = arquivo.read()
    print(contar_tokens(conteudo))
    final, un = parse_messages_from_text(conteudo)
    print(un)

    with open("nao_estruturadas.txt", "w", encoding="utf-8") as f:
        for m in un:
            f.write(m + "\n\n")

df = pd.DataFrame(final)
df.to_csv('structed.csv', index=False, encoding='utf-8')
df['mensagem_formatada'] = df['date'] + ' ' + df['time'] + ' ' + df['author'] + ": " + df['content']

contexto = ' '.join(df['mensagem_formatada'])
print(contar_tokens(contexto))
print(len(contexto.encode('utf-8'))/1024**2)

1231962


TypeError: 're.Match' object is not iterable